# Week 6 - Develop First modeling approach

In [1]:
import pandas as pd
import numpy as np
# import dask.dataframe as dd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from collections import defaultdict

In [2]:
pd.set_option('display.max_columns', None)

train = pd.read_csv('train_final.csv', low_memory=False)
validation = pd.read_csv('val_set_final.csv')

In [3]:
train.head()

,Unnamed: 0,date,customer_code,employee_index,country,female,age,new_cust,seniority_in_months,cust_type,residency_spain,birth_spain,join_channel,province_name,active_cust,income,segment,savings_acct,guarantees,current_acct,derivada_acct,payroll_acct,junior_acct,mas_particular_acct,particular_acct,particular_plus_acct,short_term_depo,medium_term_depo,long_term_depo,e_acct,funds,mortgage,pension,loans,taxes,credit_card,securities,home_acct,pensions_2,direct_debt,total_products,01 - TOP,02 - PARTICULARES,03 - UNIVERSITARIO,join_channel_encoded,province_name_encoded,employee_index_encoded,customer_code_encoded,income_to_age
0,0,2015-07-28,664160,N,1,0,0.632653,0,0.402344,1,1,0,KAR,MADRID,0,1.989686,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1.424185,1.749698,1.407278,263662,3.144938
1,1,2016-01-28,1076784,N,1,0,0.214286,0,0.152344,1,1,0,KHE,LERIDA,0,-0.306603,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0.886876,1.006139,1.407278,459750,-1.430748
2,2,2015-12-28,672465,N,1,0,0.387755,0,0.417969,1,1,0,KFC,SEVILLA,1,-0.148205,02 - PARTICULARES,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,4,0,1,0,1.559984,1.382030,1.407278,266890,-0.382203
3,3,2015-10-28,774528,N,1,0,0.397959,0,0.343750,1,1,0,KFA,MURCIA,1,-0.228531,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,1,0,1.850124,1.075147,1.407278,300528,-0.574243
4,4,2016-05-28,569598,N,1,0,0.459184,0,0.496094,1,1,0,KAT,MADRID,1,0.588748,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1.942077,1.749698,1.407278,227731,1.282133


In [4]:
validation.head()

,Unnamed: 0.1,Unnamed: 0,date,customer_code,employee_index,country_spain,female,age,first_contract_date,new_cust,seniority_in_months,primary_cust,last_date_primary,cust_type,residency_spain,birth_spain,join_channel,deceased,province_name,active_cust,income,segment,savings_acct,guarantees,current_acct,derivada_acct,payroll_acct,junior_acct,mas_particular_acct,particular_acct,particular_plus_acct,short_term_depo,medium_term_depo,long_term_depo,e_acct,funds,mortgage,pension,loans,taxes,credit_card,securities,home_acct,payroll_acct.1,pensions_2,direct_debt,total_products,01 - TOP,02 - PARTICULARES,03 - UNIVERSITARIO,join_channel_encoded,province_name_encoded,employee_index_encoded,income_to_age
0,0,0,2015-11-28,161428,N,1,1,0.744898,2000-03-16,0,0.738281,1,0,1,1,1,KAS,0,MADRID,0,-0.001810,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2.015705,1.749698,1.407278,-0.002430
1,1,1,2015-12-28,367478,N,1,1,0.418367,2002-07-02,0,0.632812,1,0,1,1,0,KAT,0,MADRID,1,1.473140,02 - PARTICULARES,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,5,0,1,0,1.942077,1.749698,1.407278,3.521079
2,2,2,2015-11-28,643150,N,1,0,0.520408,2006-09-21,0,0.429688,1,0,1,1,0,KFC,0,ALBACETE,0,-0.131613,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1.559984,1.103044,1.407278,-0.252899
3,3,3,2016-04-28,1385854,N,1,0,0.367347,2015-03-03,0,0.050781,1,0,1,1,0,KHK,0,SEVILLA,0,-0.056218,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1.122151,1.382030,1.407278,-0.153034
4,4,4,2015-08-28,495733,N,1,0,0.346939,2004-10-13,0,0.507812,1,0,1,1,0,KFC,0,MADRID,0,0.289349,02 - PARTICULARES,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,1.559984,1.749698,1.407278,0.833982


Changing columns name and dropping columns so both datasets are the same

In [5]:
train = train.rename(columns={'country': 'country_spain'})

In [6]:
train = train.drop(columns=['Unnamed: 0'])
validation = validation.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
drop = ['join_channel', 'province_name', 'employee_index', 'segment', 'total_products']
train = train.drop(columns=drop + ['customer_code_encoded'])
validation = validation.drop(columns=drop + ['payroll_acct.1', 'first_contract_date', 'primary_cust', 'last_date_primary', 'deceased'])

## Reading into the data

Setting products we want to predict

In [7]:
products = ['savings_acct', 'guarantees', 'current_acct',
       'derivada_acct', 'payroll_acct', 'junior_acct', 'mas_particular_acct',
       'particular_acct', 'particular_plus_acct', 'short_term_depo',
       'medium_term_depo', 'long_term_depo', 'e_acct', 'funds', 'mortgage',
       'pension', 'loans', 'taxes', 'credit_card', 'securities', 'home_acct',
       'pensions_2', 'direct_debt']

Dropping duplicates on customer code column since the last instance will show all the products a client has 

In [8]:
train = train.drop_duplicates(subset=['customer_code'], keep='last')
validation = validation.drop_duplicates(subset=['customer_code'], keep='last')

# Removing customers from validation set that appear in training set
validation = validation[~validation['customer_code'].isin(train['customer_code'])]

In [9]:
print(train.info())
print(validation.info())

<class 'pandas.core.frame.DataFrame'>
Index: 706866 entries, 39288 to 6579716
Data columns (total 42 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   date                    706866 non-null  object 
 1   customer_code           706866 non-null  int64  
 2   country_spain           706866 non-null  int64  
 3   female                  706866 non-null  int64  
 4   age                     706866 non-null  float64
 5   new_cust                706866 non-null  int64  
 6   seniority_in_months     706866 non-null  float64
 7   cust_type               706866 non-null  int64  
 8   residency_spain         706866 non-null  int64  
 9   birth_spain             706866 non-null  int64  
 10  active_cust             706866 non-null  int64  
 11  income                  706866 non-null  float64
 12  savings_acct            706866 non-null  int64  
 13  guarantees              706866 non-null  int64  
 14  current_acct        

# Pre-processing

Defining our Xs and Ys

In [10]:
X_train = train.drop(['customer_code', 'date'] + products, axis=1)
y_train = train[products]

X_val = validation.drop(['customer_code', 'date'] + products, axis=1)
y_val = validation[products]

In [11]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

print("Shape of X_val:", X_val.shape)
print("Shape of y_val:", y_val.shape)

Shape of X_train: (706866, 17)
Shape of y_train: (706866, 23)
Shape of X_val: (200333, 17)
Shape of y_val: (200333, 23)


## Training

In [12]:
# Hyperparameters
hyperparameter_variations = [
    {'C': 0.01, 'solver': 'liblinear', 'max_iter': 100},
    {'C': 1, 'solver': 'lbfgs', 'max_iter': 500},
    {'C': 10, 'solver': 'liblinear', 'max_iter': 300},
]

In [13]:
# Storing trained models and predictions
models = {}
metrics = defaultdict(lambda: defaultdict(dict))

We will create a model to train on the training data using all 3 hyperparameters we set. We will use this trained model to predict the product recommendations on the validation set and compare the results between the different hyperparameters and different metrics we chose to use, which are ROC AUC, F1 Score and Confusion Matrix.  
We will calculate ROC AUC using probabilities (predict_proba() method), which is more appropriate for this metric since ROC AUC works with predicted probabilities for the positive class and not binary predictions.  
F1 Score and confusion matrix were calculated using the binary predictions (predict() method), which is the correct approach for these metrics.

In [14]:
# Train and evaluate each hyperparameter variation
for i, params in enumerate(hyperparameter_variations):
    print(f"\nTraining variation {i + 1} with parameters: {params}")
    
    for product in products:
        print(f"Training model for product: {product}")
        clf = LogisticRegression(**params)
        
        # Target column for current product
        y_train_product = y_train[product].values
        y_val_product = y_val[product].values
        
        # Train the model on current product
        clf.fit(X_train, y_train_product)
        
        # Make predictions
        y_train_pred = clf.predict(X_train)
        y_val_pred = clf.predict(X_val)
        y_train_pred_proba = clf.predict_proba(X_train)[:, 1]
        y_val_pred_proba = clf.predict_proba(X_val)[:, 1]
        
        # Calculate metrics for training set and validation sets
        metrics[f'Variation {i + 1}']['train'][product] = {
            'ROC AUC': roc_auc_score(y_train_product, y_train_pred_proba),
            'F1 Score': f1_score(y_train_product, y_train_pred),
            'Confusion Matrix': confusion_matrix(y_train_product, y_train_pred)
        }
        
        metrics[f'Variation {i + 1}']['val'][product] = {
            'ROC AUC': roc_auc_score(y_val_product, y_val_pred_proba),
            'F1 Score': f1_score(y_val_product, y_val_pred),
            'Confusion Matrix': confusion_matrix(y_val_product, y_val_pred)
        }
        
        print(f"\nResults for'{product}' in variation {i + 1}:")
        print(f"Training - ROC AUC: {metrics[f'Variation {i + 1}']['train'][product]['ROC AUC']:.4f}, "
              f"F1 Score: {metrics[f'Variation {i + 1}']['train'][product]['F1 Score']:.4f}")
        print(f"Validation - ROC AUC: {metrics[f'Variation {i + 1}']['val'][product]['ROC AUC']:.4f}, "
              f"F1 Score: {metrics[f'Variation {i + 1}']['val'][product]['F1 Score']:.4f}")


Training variation 1 with parameters: {'C': 0.01, 'solver': 'liblinear', 'max_iter': 100}
Training model for product: savings_acct

Results for'savings_acct' in variation 1:
Training - ROC AUC: 0.4029, F1 Score: 0.0000
Validation - ROC AUC: 0.4489, F1 Score: 0.0000
Training model for product: guarantees

Results for'guarantees' in variation 1:
Training - ROC AUC: 0.2619, F1 Score: 0.0000
Validation - ROC AUC: 0.1172, F1 Score: 0.0000
Training model for product: current_acct

Results for'current_acct' in variation 1:
Training - ROC AUC: 0.7485, F1 Score: 0.7865
Validation - ROC AUC: 0.7564, F1 Score: 0.7899
Training model for product: derivada_acct

Results for'derivada_acct' in variation 1:
Training - ROC AUC: 0.7772, F1 Score: 0.0000
Validation - ROC AUC: 0.7503, F1 Score: 0.0000
Training model for product: payroll_acct

Results for'payroll_acct' in variation 1:
Training - ROC AUC: 0.8646, F1 Score: 0.0001
Validation - ROC AUC: 0.8595, F1 Score: 0.0075
Training model for product: jun

c:\Users\MARIA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Results for'current_acct' in variation 2:
Training - ROC AUC: 0.7480, F1 Score: 0.7863
Validation - ROC AUC: 0.7569, F1 Score: 0.7895
Training model for product: derivada_acct


c:\Users\MARIA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Results for'derivada_acct' in variation 2:
Training - ROC AUC: 0.8804, F1 Score: 0.0000
Validation - ROC AUC: 0.9080, F1 Score: 0.0000
Training model for product: payroll_acct


...


Results for'home_acct' in variation 2:
Training - ROC AUC: 0.8889, F1 Score: 0.0000
Validation - ROC AUC: 0.9137, F1 Score: 0.0091
Training model for product: pensions_2


c:\Users\MARIA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Results for'pensions_2' in variation 2:
Training - ROC AUC: 0.8623, F1 Score: 0.0013
Validation - ROC AUC: 0.8596, F1 Score: 0.0105
Training model for product: direct_debt


c:\Users\MARIA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Results for'direct_debt' in variation 2:
Training - ROC AUC: 0.8691, F1 Score: 0.0500
Validation - ROC AUC: 0.8709, F1 Score: 0.0606

Training variation 3 with parameters: {'C': 10, 'solver': 'liblinear', 'max_iter': 300}
Training model for product: savings_acct

Results for'savings_acct' in variation 3:
Training - ROC AUC: 0.8729, F1 Score: 0.0000
Validation - ROC AUC: 0.9315, F1 Score: 0.0000
Training model for product: guarantees

Results for'guarantees' in variation 3:
Training - ROC AUC: 0.9734, F1 Score: 0.0000
Validation - ROC AUC: 0.9106, F1 Score: 0.0000
Training model for product: current_acct

Results for'current_acct' in variation 3:
Training - ROC AUC: 0.7490, F1 Score: 0.7866
Validation - ROC AUC: 0.7587, F1 Score: 0.7901
Training model for product: derivada_acct

Results for'derivada_acct' in variation 3:
Training - ROC AUC: 0.8808, F1 Score: 0.0000
Validation - ROC AUC: 0.8979, F1 Score: 0.0000
Training model for product: payroll_acct

Results for'payroll_acct' in vari

Creating a table to see the results in a easier to interpret way

In [15]:
train_metrics_df = pd.DataFrame.from_dict({(i,j): metrics[i]['train'][j] 
                     for i in metrics.keys() 
                     for j in products},
                    orient='index')

val_metrics_df = pd.DataFrame.from_dict({(i,j): metrics[i]['val'][j] 
                     for i in metrics.keys() 
                     for j in products},
                    orient='index')

In [16]:
pd.set_option('display.max_rows', None)
print("Training Metrics Table:")
train_metrics_df

Training Metrics Table:


ROC AUC  F1 Score  \
Variation 1 savings_acct          0.402916  0.000000   
            guarantees            0.261854  0.000000   
            current_acct          0.748522  0.786467   
            derivada_acct         0.777204  0.000000   
            payroll_acct          0.864570  0.000103   
            junior_acct           0.998986  0.141183   
            mas_particular_acct   0.825569  0.000000   
            particular_acct       0.884666  0.214141   
            particular_plus_acct  0.813231  0.000000   
            short_term_depo       0.924857  0.000000   
            medium_term_depo      0.877228  0.000000   
            long_term_depo        0.926317  0.341200   
            e_acct                0.860581  0.216472   
            funds                 0.921595  0.000000   
            mortgage              0.920838  0.000000   
            pension               0.920788  0.000000   
            loans                 0.839712  0.000000   
            taxes                 0.859193  0.000784   
            credit_card           0.890356  0.000129   
            securities            0.913740  0.000227   
            home_acct             0.871426  0.000000   
            pensions_2            0.861401  0.000094   
            direct_debt           0.868809  0.035647   
Variation 2 savings_acct          0.868903  0.000000   
            guarantees            0.964197  0.000000   
            current_acct          0.748034  0.786279   
            derivada_acct         0.880394  0.000000   
            payroll_acct          0.866318  0.000256   
            junior_acct           0.999425  0.881061   
            mas_particular_acct   0.841209  0.000000   
            particular_acct       0.884820  0.229081   
            particular_plus_acct  0.813776  0.000000   
            short_term_depo       0.945518  0.000000   
            medium_term_depo      0.895236  0.000000   
            long_term_depo        0.927018  0.350823   
            e_acct                0.860760  0.221820   
            funds                 0.923179  0.004361   
            mortgage              0.926707  0.000000   
            pension               0.922319  0.005588   
            loans                 0.854446  0.000000   
            taxes                 0.859697  0.001202   
            credit_card           0.890593  0.006080   
            securities            0.914192  0.007435   
            home_acct             0.888943  0.000000   
            pensions_2            0.862302  0.001266   
            direct_debt           0.869138  0.049997   
Variation 3 savings_acct          0.872940  0.000000   
            guarantees            0.973446  0.000000   
            current_acct          0.748993  0.786626   
            derivada_acct         0.880811  0.000000   
            payroll_acct          0.866259  0.001025   
            junior_acct           0.999570  0.885389   
            mas_particular_acct   0.841194  0.000000   
            particular_acct       0.884887  0.230861   
            particular_plus_acct  0.813682  0.000000   
            short_term_depo       0.947280  0.000000   
            medium_term_depo      0.895238  0.000000   
            long_term_depo        0.927018  0.350078   
            e_acct                0.861151  0.220272   
            funds                 0.923120  0.004206   
            mortgage              0.927031  0.000000   
            pension               0.922354  0.005587   
            loans                 0.854412  0.000000   
            taxes                 0.859777  0.001202   
            credit_card           0.890683  0.006335   
            securities            0.914217  0.007435   
            home_acct             0.889242  0.000000   
            pensions_2            0.862506  0.001032   
            direct_debt           0.869315  0.046836   

                                                     Confusion Matrix  
Variation 1 savings_acct                  

In [17]:
print("Validation Metrics Table:")
val_metrics_df

Validation Metrics Table:


ROC AUC  F1 Score  \
Variation 1 savings_acct          0.448868  0.000000   
            guarantees            0.117191  0.000000   
            current_acct          0.756396  0.789899   
            derivada_acct         0.750282  0.000000   
            payroll_acct          0.859517  0.007521   
            junior_acct           0.997220  0.127197   
            mas_particular_acct   0.886404  0.000000   
            particular_acct       0.926977  0.245744   
            particular_plus_acct  0.858969  0.028845   
            short_term_depo       0.912585  0.000000   
            medium_term_depo      0.897696  0.000000   
            long_term_depo        0.935919  0.293059   
            e_acct                0.876874  0.168256   
            funds                 0.935718  0.058437   
            mortgage              0.946640  0.000000   
            pension               0.932103  0.037847   
            loans                 0.858123  0.000000   
            taxes                 0.865688  0.030680   
            credit_card           0.918365  0.040483   
            securities            0.928463  0.085966   
            home_acct             0.899709  0.000000   
            pensions_2            0.858688  0.009136   
            direct_debt           0.870896  0.051411   
Variation 2 savings_acct          0.933433  0.000000   
            guarantees            0.988746  0.000000   
            current_acct          0.756892  0.789503   
            derivada_acct         0.907970  0.000000   
            payroll_acct          0.860161  0.008951   
            junior_acct           0.998574  0.827032   
            mas_particular_acct   0.885058  0.000000   
            particular_acct       0.925391  0.253438   
            particular_plus_acct  0.861638  0.029767   
            short_term_depo       0.945722  0.005831   
            medium_term_depo      0.927063  0.037736   
            long_term_depo        0.938429  0.303536   
            e_acct                0.873791  0.171671   
            funds                 0.938061  0.061080   
            mortgage              0.948643  0.005950   
            pension               0.933830  0.031701   
            loans                 0.887794  0.000000   
            taxes                 0.865983  0.031130   
            credit_card           0.920199  0.044042   
            securities            0.930085  0.096486   
            home_acct             0.913726  0.009112   
            pensions_2            0.859630  0.010488   
            direct_debt           0.870889  0.060638   
Variation 3 savings_acct          0.931511  0.000000   
            guarantees            0.910590  0.000000   
            current_acct          0.758703  0.790143   
            derivada_acct         0.897898  0.000000   
            payroll_acct          0.864173  0.008957   
            junior_acct           0.999552  0.840149   
            mas_particular_acct   0.890196  0.000000   
            particular_acct       0.930324  0.257369   
            particular_plus_acct  0.866979  0.000000   
            short_term_depo       0.943757  0.012579   
            medium_term_depo      0.928459  0.035794   
            long_term_depo        0.940121  0.305180   
            e_acct                0.879096  0.170943   
            funds                 0.942005  0.059712   
            mortgage              0.951792  0.007890   
            pension               0.939714  0.036748   
            loans                 0.887644  0.000000   
            taxes                 0.873381  0.023401   
            credit_card           0.923466  0.043750   
            securities            0.931958  0.097025   
            home_acct             0.917353  0.000000   
            pensions_2            0.861479  0.010707   
            direct_debt           0.870459  0.058622   

                                                   Confusion Matrix  
Variation 1 savings_acct                    

Creating a summary table for all the variations and different datasets

In [18]:
summary_data = []
for variation in metrics:
    for dataset in ['train', 'val']:
        avg_roc_auc = np.mean([metrics[variation][dataset][p]['ROC AUC'] for p in products])
        avg_f1 = np.mean([metrics[variation][dataset][p]['F1 Score'] for p in products])
        summary_data.append([variation, dataset, avg_roc_auc, avg_f1])

summary_df = pd.DataFrame(summary_data, columns=['Variation', 'Dataset', 'Avg ROC AUC', 'Avg F1 Score'])
print("Summary Table:")
print(summary_df.to_string(index=False))

best_variation = summary_df[summary_df['Dataset'] == 'val'].sort_values('Avg ROC AUC', ascending=False).iloc[0]['Variation']
print(f"\nBest Model For This Week: {best_variation}")

Summary Table:
  Variation Dataset  Avg ROC AUC  Avg F1 Score
Variation 1   train     0.827581      0.075498
Variation 1     val     0.836491      0.085847
Variation 2   train     0.887266      0.110663
Variation 2     val     0.907466      0.120787
Variation 3   train     0.888049      0.110734
Variation 3     val     0.906114      0.119955

Best Model For This Week: Variation 2


## Generate product recommendations

Here we want to visualize the product recommendations for each customer 

In [19]:
best_params = hyperparameter_variations[int(best_variation.split()[-1]) - 1]

# Train models and predict for each product for all customers using parameters from Variation 3
product_models = {}
for product in products:
    clf = LogisticRegression(**best_params)
    clf.fit(X_train, y_train[product])
    product_models[product] = clf

train_preds = {}
for product in products:
    proba = product_models[product].predict_proba(X_train)[:, 1]
    for customer_id, prob in zip(train['customer_code'], proba):
        if customer_id not in train_preds:
            train_preds[customer_id] = []
        train_preds[customer_id].append((product, prob))

c:\Users\MARIA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\MARIA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

We want to make sure that we are not recommending a product that the customer already own, so we will store the products that customers already have

In [20]:
def get_active_products(customer_data):
    return set(product for product in products if customer_data[product] > 0)

We will sort the recommended products by the probability of a client getting it and we will get the top 7 recommendations

In [21]:
for customer_id in train_preds:
    # Sorting by probability
    sorted_prods = sorted(train_preds[customer_id], key=lambda x: x[1], reverse=True)
    customer_data = train[train['customer_code'] == customer_id].iloc[0]
    # Filter out already active products
    active_products = get_active_products(customer_data)
    recommended_products = [prod for prod, _ in sorted_prods if prod not in active_products]
    
    # Get top 7
    train_preds[customer_id] = recommended_products[:7]

# Example recommendations
print("Recommendations:")
for customer_id in list(train_preds.keys())[:5]:
    print(f"Customer {customer_id}: {train_preds[customer_id]}")

Recommendations:
Customer 1225385: ['current_acct', 'e_acct', 'particular_acct', 'taxes', 'particular_plus_acct', 'mas_particular_acct', 'direct_debt']
Customer 1358829: ['direct_debt', 'payroll_acct', 'pensions_2', 'e_acct', 'taxes', 'credit_card', 'long_term_depo']
Customer 1436539: ['direct_debt', 'payroll_acct', 'pensions_2', 'e_acct', 'short_term_depo', 'taxes', 'credit_card']
Customer 1448049: ['current_acct', 'e_acct', 'taxes', 'mas_particular_acct', 'direct_debt', 'home_acct', 'payroll_acct']
Customer 1396837: ['direct_debt', 'payroll_acct', 'pensions_2', 'mas_particular_acct', 'e_acct', 'junior_acct', 'long_term_depo']


Lastly, we want to identify which product has been recommended the most and least in the model

In [22]:
product_rec_counts = {product: sum(1 for recs in train_preds.values() if product in recs) for product in products}
most_rec = max(product_rec_counts, key=product_rec_counts.get)
least_rec = min(product_rec_counts, key=product_rec_counts.get)

print(f"Most frequently recommended product: {most_rec} ({product_rec_counts[most_rec]} times)")
print(f"Least frequently recommended product: {least_rec} ({product_rec_counts[least_rec]} times)")

Most frequently recommended product: taxes (652073 times)
Least frequently recommended product: savings_acct (2 times)
